# dbachecks PR check on Beard NUC - MultipleInstances

Rob uses this process to make sure that changes will work in an AD estate with a variety of versions

First we must check the status of the local source control and ensure that we are in the PR branch

In [1]:
cd D:\OneDrive\Documents\GitHub\dbachecks
    git status
    

On branch pr/jpomfret/768-1


Your branch is up to date with 'jpomfret/suspectpagelimit'.


nothing to commit, working tree clean


First we need to import the module from source control - I normally use verbose to see how it is going


In [2]:
Import-Module .\dbachecks.psd1 -Verbose
$null= Reset-DbcConfig 

VERBOSE: Loading module from path 'D:\OneDrive\Documents\GitHub\dbachecks\dbachecks.psd1'.


VERBOSE: Loading 'FormatsToProcess' from path 'D:\OneDrive\Documents\GitHub\dbachecks\xml\dbachecks.Format.ps1xml'.


VERBOSE: Loading module from path 'D:\OneDrive\Documents\GitHub\dbachecks\dbachecks.psm1'.


VERBOSE: Exporting function 'Import-ModuleFile'.


VERBOSE: Exporting function 'Get-CheckFile'.


...

VERBOSE: Importing function 'Save-DbcRequiredModules'.


VERBOSE: Importing function 'Set-DbcConfig'.


VERBOSE: Importing function 'Start-DbcPowerBi'.


VERBOSE: Importing function 'Update-DbcPowerBiDataSource'.


VERBOSE: Importing function 'Update-DbcRequiredModules'.


Excellent, that worked. Now to run some checks against a single instance  

I don't care if the test pass or fail, I am only interested in any 

> Describe block has Error

or

> Error in Script

type failures

Multiple Instances at parameter

# Make sure that the check and config have been written to the json file

In [3]:
Get-DbcCheck suspectpagelimit | fl

Group       : Instance


Type        : Sqlinstance


UniqueTag   : SuspectPageLimit


AllTags     : SuspectPageLimit, Medium, Instance


Config      : app.sqlinstance skip.instance.suspectpagelimit policy.suspectpages.threshold 


Description : Tests that the suspect_pages table in msdb is not nearing the limit of 1000 rows


In [5]:
Get-DbcConfig skip.instance.suspectpagelimit   | fl
Get-DbcConfig policy.suspectpages.threshold   | fl

Name        : skip.instance.suspectpagelimit


Value       : False


Description : Skip the check for whether the suspect_pages table is nearing the row limit of 1000


Name        : policy.suspectpages.threshold


Value       : 90


Description : Default threshold (%) to check whether suspect_pages is nearing row limit of 1000


I had already checked both AGs with dbachecks HADR check to ensure that they were up and running correctly

In [8]:
$Sqlinstances = 'SQL2012Ser08AG1','SQL2012Ser08AG2','SQL2012Ser08AG3','SQL2016N1','SQL2016N2','SQL2016N3'
Set-DbcConfig -Name skip.backup.readonly -Value $true
Invoke-DbcCheck -SqlInstance $Sqlinstances -Check LastFullBackup 

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1' with Tags LastFullBackup


Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1


  Describing Last Full Backup Times


    Context Testing last full backups on SQL2012Ser08AG1


      [+] Database AdventureWorks2012AG should have full backups less than 1 days old on SQL2012Ser08AG1

...

Pending: 0, 

Inconclusive: 0 


Name                 Value Description


----                 ----- -----------


skip.backup.readonly  True Check read-only databases for last backup


Set the config and run again

In [9]:
$InstanceSMO = Connect-DbaInstance SQL2016N2
$skipsecondaries = $true
$InstanceSMO.Databases.Where{ ($psitem.Name -ne 'tempdb') }.ForEach{
    if ($psitem.AvailabilityGroupName) {
        $agReplicaRole = $InstanceSMO.AvailabilityGroups[$psitem.AvailabilityGroupName].LocalReplicaRole
    } else {
        $agReplicaRole = $null
    }
    $skip = ($psitem.Status -match "Offline") -or ($psitem.IsAccessible -eq $false) -or ($psitem.Readonly -eq $true -and $skipreadonly -eq $true) -or ($agReplicaRole -eq 'Secondary' -and $skipsecondaries -eq $true)
Write-Host " $agReplicaRole and $($Psitem.Name) and $skip"
}

  and DBA-Admin and False


  and dbachecks and False


 Secondary and DEMOdbareports and True


 Secondary and Lisbondbareports and True


  and master and False


...

 Secondary and passrocssug_dbareports and True


  and SQLWatch and False


 Secondary and TheBeardsProd and True


 Secondary and VideoDemodbareports and True


 Secondary and Viennadbareports and True


In [17]:
$Sqlinstances = (Get-VM -ComputerName BEARDNUC, BEARDNUC2 -Name *SQL* | Where State -eq 'Running').Name

Set-DbcConfig -Name skip.security.nonstandardport -Value $false
Invoke-DbcCheck -ComputerName SQL2017N5 -Check NonStandardPort


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Server.Tests.ps1' with Tags NonStandardPort

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Server.Tests.ps1

  Describing Non Standard Port

    Context Checking SQL Server ports on SQL2017N5
      [+] No SQL Server Instances should be configured with port 1433 on SQL2017N5 3ms
Tests completed in 281ms
Tests Passed: 1, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 
Name                          Value Description
----                          ----- -----------
skip.security.nonstandardport False Skips the check for whether SQL Server should be configured wi…



In [15]:
Get-DbcConfig -Name global.notcontactable 


Name                  Value Description
----                  ----- -----------
global.notcontactable {}    This is used within the checks to avoid trying to contact none-respons…



In [9]:
@(Get-DbaLogin -SQLInstance sql2019n5 -Type SQL | Where-Object { $_.PasswordPolicyEnforced -eq $false -and $_.IsDisabled -eq $false}).Count

1


In [10]:
@(Get-DbaLogin -SqlInstance sql2019n5 -Login sa).Count

1
